In [1]:
import pandas as pd
from eunjeon import Mecab

### 데이터 불러오기

In [4]:
정상기업뉴스 = pd.read_csv('../Step3-1_뉴스전처리/전처리후/정상기업뉴스전처리.csv', encoding='cp949')
정상기업뉴스추가 = pd.read_csv('../Step3-1_뉴스전처리/전처리후/정상기업뉴스전처리추가.csv')
정상기업 = pd.concat([정상기업뉴스, 정상기업뉴스추가])
정상기업.reset_index(inplace=True, drop=True)

### 전처리된 뉴스기사본문 토큰화

In [6]:
mecab=Mecab()
## 형태소 분석기 mecab를 사용한 토큰화 작업 (다소 시간 소요)

## Mecab으로 토큰화
정상기업['뉴스기사본문전처리'] = 정상기업['뉴스기사본문전처리'].apply(lambda x: mecab.nouns(x))
## 명사만 추출
정상기업['뉴스기사본문전처리'] = 정상기업['뉴스기사본문전처리'].apply(lambda x: [word for word in x if len(word) > 1])

### 분류단어 넣기

In [7]:
분류단어 = ['폐지', '격성', '퇴출', '실질', '심사', '속개', '이의', '심의', '거절']
분류단어 = '|'.join(분류단어)
분류단어

'폐지|격성|퇴출|실질|심사|속개|이의|심의|거절'

### 부도기사 레이블화

In [8]:
라벨 = 정상기업['뉴스기사본문전처리'].apply(lambda x: [word for word in x if word in 분류단어]) # 분류단어 들어간 것들
라벨 = 라벨.apply(lambda x: 1 if (len(x)>=1) else 0)
정상기업['뉴스기사 레이블'] = 라벨

### 기업 당 뉴스기사 수 추출(정상기업에서는 기업이 연도별로 다른 것으로 생각하므로 groupby할때 연도와 같이 묶어줘야함)

### 정상기업에 대해서 연도별로 묶기위해 컬럼 도출

In [21]:
time=정상기업["기사발행일"].str.split('.').tolist()
time_df=pd.DataFrame(time,columns=['년','월','일','시간'])
time_df.drop("시간",axis=1,inplace=True)

time_data=pd.concat([time_df,정상기업],axis=1)
time_data.drop(["월","일","기사발행일"],axis=1,inplace=True)

기사수 = time_data.groupby(["기업","년"]).count()

### 기업 당 뉴스 부도뉴스 레이블 합 도출

In [22]:
레이블합=time_data.groupby(["기업","년"]).sum()

### 부도기사비율 도출

In [23]:
기사수["부도기사비율"]=기사수["기사제목"]
기사수["부도기사비율"]=레이블합["뉴스기사 레이블"]/기사수["뉴스기사 레이블"]*100
기사수.reset_index(inplace=True)
기사수.drop(['기사제목', '뉴스기사본문전처리', '뉴스기사 레이블'], axis = 1, inplace=True)
기사수.to_csv('./datasets/정상기업_부도기사 비율.csv', index=None)

,기업,년,부도기사비율
0,경창산업,2013,8.064516
1,고영테크놀러지,2011,19.148936
2,광림,2016,1.818182
3,금화피에스시,2013,16.901408
4,나스미디어,2017,1.449275
...,...,...,...
119,하이록코리아,2012,1.408451
120,한국정보통신,2015,4.545455
121,해성산업,2013,9.523810
122,홈캐스트,2016,8.695652
